# Контрольная работа
## Вариант 60
### Выполнил: студент ПИ19-3 Ковалев А.И.

## Задание 1

In [1]:
from functools import lru_cache
import numpy as np


def transition_probability(matrix, k):
    """Вероятность перехода из x в y за k шагов"""
    return np.linalg.matrix_power(matrix, k)


def state_probability(matrix, start, k):
    """Вероятность состояния за k шагов"""
    return start.dot(np.linalg.matrix_power(matrix, k))


def first_transition_probability(matrix, k):
    """Вероятность первого перехода за k шагов"""
    prev = np.copy(matrix)
    for _ in range(k - 1):
        prev = matrix_power_with_skip(matrix, prev)

    return prev


def last_transition_probability(matrix, k):
    """Вероятность перехода не позднее чем за k шагов"""
    prev, res = np.copy(matrix), np.copy(matrix)
    for t in range(k - 1):
        prev = matrix_power_with_skip(matrix, prev)
        res += prev

    return res


def avg_steps(matrix):
    """Среднее количество шагов, необходимых для первого перехода"""
    prev, res = np.copy(matrix), np.copy(matrix)
    for t in range(993):
        prev = matrix_power_with_skip(matrix, prev)
        res += t * prev

    return res


def first_return_probability(matrix, k):
    """Вероятность первого возвращения на k-ом шаге"""
    _matrix = np.copy(matrix)
    p_jj = transition_probability

    @lru_cache(maxsize=None)
    def f_jj(_k):
        return p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])

    return np.diagonal(f_jj(k))


def avg_time_return(matrix):
    """Среднее время возвращения"""
    _matrix = np.copy(matrix)
    p_jj = transition_probability
    result = []

    @lru_cache(maxsize=None)
    def f_jj(_k):
        res = p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])
        result.append(_k * np.diagonal(res))
        return res

    f_jj(993)
    return sum(result)


def last_return_probability(matrix, k):
    """Вероятность возвращения не позднее чем за k шагов"""
    _matrix = np.copy(matrix)
    p_jj = transition_probability
    result = []

    @lru_cache(maxsize=None)
    def f_jj(_k):
        res = p_jj(_matrix, _k) - sum([f_jj(m) * p_jj(_matrix, _k - m) for m in range(1, _k)])
        result.append(np.diagonal(res))
        return res

    f_jj(k)
    return sum(result)


def steady_state_probabilities(matrix):
    """Установившиеся вероятности"""
    matrix_ = np.copy(matrix).transpose()
    np.fill_diagonal(matrix_, np.diagonal(matrix_) - 1)
    matrix_[-1, :] = 1

    vec_b = np.zeros(len(matrix_))
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)


def matrix_power_with_skip(left, right):
    range_ = range(len(left))
    return np.array([[sum(left[i, m] * right[m, j] if m != j else 0 for m in range_) for j in range_] for i in range_])


def validate(matrix):
    assert matrix.shape == (13, 13), "Размер матрицы должен быть (13, 13)"
    assert np.equal(np.sum(matrix, axis=1), np.matrix(np.ones(13))).all(), (
        "Сумма вероятностей в каждой строке должна равняться 1"
    )
    assert (np.count_nonzero(transition_matrix, axis=1) ==
            np.array([3, 3, 3, 3, 1, 2, 4, 5, 2, 2, 4, 3, 1]) + 1).all(), (
        "Количество исходящих вероятностей должно быть [3, 3, 3, 3, 1, 2, 4, 5, 2, 2, 4, 3, 1]"
    )


def answer(res):
    return f"Ответ: {res}\n"

Система имеем 13 дискретных состояний. Изменение состояний происходит
в дискретные моменты времени с заданной вероятность. Схема марковского
процесса изображена на рисунке. Требуется определить:

In [2]:
transition_matrix = np.array([
    [0.09, 0.56, 0, 0, 0.34, 0.01, 0, 0, 0, 0, 0, 0, 0],
    [0.47, 0.3, 0, 0.18, 0.05, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0.31, 0.11, 0, 0.33, 0.25, 0, 0, 0, 0, 0, 0, 0],
    [0.24, 0.2, 0, 0.31, 0, 0, 0.25, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0.39, 0.61, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0.23, 0, 0.49, 0.28, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0.34, 0.24, 0, 0.04, 0, 0, 0, 0.35, 0.03, 0],
    [0, 0, 0, 0, 0.26, 0, 0.2, 0.22, 0.28, 0.02, 0.02, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0.07, 0.61, 0.32, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0.69, 0, 0.24, 0.07, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0.3, 0, 0, 0.31, 0.03, 0.13, 0.23],
    [0, 0, 0, 0, 0, 0, 0.59, 0.05, 0, 0, 0.29, 0.07, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0.87, 0, 0, 0.13]
])
validate(transition_matrix)

print("Матрица переходов:")
print(transition_matrix, "\n")

Матрица переходов:
[[0.09 0.56 0.   0.   0.34 0.01 0.   0.   0.   0.   0.   0.   0.  ]
 [0.47 0.3  0.   0.18 0.05 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.31 0.11 0.   0.33 0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.24 0.2  0.   0.31 0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.39 0.61 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.23 0.   0.49 0.28 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.34 0.24 0.   0.04 0.   0.   0.   0.35 0.03 0.  ]
 [0.   0.   0.   0.   0.26 0.   0.2  0.22 0.28 0.02 0.02 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.07 0.61 0.32 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.69 0.   0.24 0.07 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.3  0.   0.   0.31 0.03 0.13 0.23]
 [0.   0.   0.   0.   0.   0.   0.59 0.05 0.   0.   0.29 0.07 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.87 0.   0.   0.13]] 



In [3]:
k, i, j = 10, 10, 1
transition = transition_probability(transition_matrix, k)
print(f"1. вероятность того, что за {k} шагов система перейдет из состояния {i} в состояние {j}:")
print(answer(transition[i - 1, j - 1]))

1. вероятность того, что за 10 шагов система перейдет из состояния 10 в состояние 1:
Ответ: 0.13106361683537357



In [4]:
k = 10
a_0 = np.array([0, 0.03, 0.08, 0.14, 0.16, 0.04, 0.13, 0.11, 0.07, 0.11, 0.01, 0.06, 0.06])
state = state_probability(transition_matrix, a_0, k)
print(f"2. вероятности состояний системы спустя {k} шагов, если в начальный "
      f"момент вероятность состояний были следующими A={a_0}:")
print(answer(state))

2. вероятности состояний системы спустя 10 шагов, если в начальный момент вероятность состояний были следующими A=[0.   0.03 0.08 0.14 0.16 0.04 0.13 0.11 0.07 0.11 0.01 0.06 0.06]:
Ответ: [0.14497862 0.17428118 0.00054258 0.21791259 0.21524969 0.00214426
 0.10625754 0.00397231 0.03290224 0.03831057 0.04228991 0.00960522
 0.01155329]



In [5]:
k, i, j = 9, 11, 5
first_transition = first_transition_probability(transition_matrix, k)
print(f"3. вероятность первого перехода за {k} шагов из состояния {i} в состояние {j}:")
print(answer(first_transition[i - 1, j - 1]))

3. вероятность первого перехода за 9 шагов из состояния 11 в состояние 5:
Ответ: 0.05280472542103962



In [6]:
k, i, j = 6, 6, 3
last_transition = last_transition_probability(transition_matrix, k)
print(f"4. вероятность перехода из состояния {i} в состояние {j} не позднее чем за {k} шагов:")
print(answer(last_transition[i - 1, j - 1]))

4. вероятность перехода из состояния 6 в состояние 3 не позднее чем за 6 шагов:
Ответ: 0.319602925188



In [7]:
i, j = 7, 10
avg_steps_number = avg_steps(transition_matrix)
print(f"5. среднее количество шагов для перехода из состояния {i} в состояние {j}:")
print(answer(avg_steps_number[i - 1, j - 1]))

5. среднее количество шагов для перехода из состояния 7 в состояние 10:
Ответ: 44.98666795391084



In [8]:
k, i = 7, 9
first_return = first_return_probability(transition_matrix, k)
print(f"6. вероятность первого возвращения в состояние {i} за {k} шагов:")
print(answer(first_return[i - 1]))

6. вероятность первого возвращения в состояние 9 за 7 шагов:
Ответ: 0.00344614899871995



In [9]:
k, i = 8, 12
last_return = last_return_probability(transition_matrix, k)
print(f"7. вероятность возвращения в состояние {i} не позднее чем за {k} шагов")
print(answer(last_return[i - 1]))

7. вероятность возвращения в состояние 12 не позднее чем за 8 шагов
Ответ: 0.2111283959108131



In [10]:
i = 7
avg_time_to_return = avg_time_return(transition_matrix)
print(f"8. среднее время возвращения в состояние {i}")
print(answer(avg_time_to_return[i - 1]))

8. среднее время возвращения в состояние 7
Ответ: 10.45705337814896



In [11]:
steady_state = steady_state_probabilities(transition_matrix)
print("9. установившиеся вероятности:")
print(answer(steady_state))

9. установившиеся вероятности:
Ответ: [0.15736405 0.19021431 0.0006205  0.22416898 0.22544637 0.00240106
 0.09562923 0.00222035 0.01884122 0.02802657 0.03702058 0.00825973
 0.00978705]



## Задание 2

In [12]:
import numpy as np


def steady_state_probability(matrix):
    """Установившиеся вероятности"""
    size = len(matrix)
    matrix_ = np.copy(matrix).transpose()
    np.fill_diagonal(matrix_, [-sum(matrix_[:, i]) for i in range(size)])
    matrix_[-1, :] = 1

    vec_b = np.zeros(size)
    vec_b[-1] = 1
    return np.linalg.inv(matrix_).dot(vec_b)


def failure_probability(s_state):
    """Вероятность отказа"""
    return s_state[-1]


def relative_and_absolute_service_intensity(s_state, la):
    """Относительная и абсолютная интенсивность обслуживания"""
    relative = 1 - s_state[-1]
    return relative, relative * la


def average_queue_length(s_state, m, n):
    """Средняя длина очереди"""
    return sum((i * s_state[m + i]) for i in range(1, n + 1))


def average_queue_time(s_state, m, n, mu):
    """Среднее время в очереди"""
    return sum(((i + 1) / (m * mu) * s_state[m + i]) for i in range(n))


def average_number_busy_channels(s_state, m, n):
    """Среднее число занятых каналов"""
    return (sum((i * s_state[i]) for i in range(1, m + 1)) +
            sum((m * s_state[i]) for i in range(m + 1, m + n + 1)))


def skip_queue_probability(s_state, m):
    """Вероятность не ждать в очереди"""
    return sum(s_state[:m])


def average_downtime(matrix):
    """Среднее время простоя системы массового обслуживания"""
    return 1 / np.sum(matrix, axis=1)


def init_matrix(n, m, la, mu):
    size = n + m + 1
    matrix = np.zeros((size, size))
    np.fill_diagonal(matrix[:, 1:], la)
    np.fill_diagonal(matrix[1:, :], [*[i * mu for i in range(1, m)], *[m * mu for _ in range(n + 1)]])
    return matrix


def validate(matrix):
    size = n + m + 1
    assert matrix.shape == (size, size), f"Размер матрицы должен быть {size}"


def answer(*args):
    res = ", ".join([str(i) for i in args])
    return f"Ответ: {res}\n"

Задана система массового обслуживания со следующими характеристиками:
- интенсивность поступления λ=32
- каналов обслуживания m=5
- интенсивность обслуживания μ=8
- максимальный размер очереди n=17

Изначально требований в системе нет.

<img src="resources/2_a.png">

In [13]:
la = 32
m = 5
mu = 8
n = 17
initial_matrix = init_matrix(n, m, la, mu)
validate(initial_matrix)
initial_matrix

array([[ 0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 8.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 16.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 24.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 32.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 40.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 40.,  0., 32.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 40.,  0., 32.,  0.,  0.,  0.,  0.,
   

In [14]:
steady_state = steady_state_probability(initial_matrix)
print("a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и "
      "найдите установившиеся вероятности состояний:")
print(answer(steady_state))

a) Составьте граф марковского процесса, запишите систему уравнений Колмогорова и найдите установившиеся вероятности состояний:
Ответ: [0.01311796 0.05247183 0.10494365 0.13992487 0.13992487 0.1119399
 0.08955192 0.07164153 0.05731323 0.04585058 0.03668046 0.02934437
 0.0234755  0.0187804  0.01502432 0.01201945 0.00961556 0.00769245
 0.00615396 0.00492317 0.00393853 0.00315083 0.00252066]



In [15]:
fail_probability = failure_probability(steady_state)
print("b) Найдите вероятность отказа в обслуживании:")
print(answer(fail_probability))

b) Найдите вероятность отказа в обслуживании:
Ответ: 0.0025206623548613455



In [16]:
relative, absolute = relative_and_absolute_service_intensity(steady_state, la)
print("c) Найдите относительную и абсолютную интенсивность обслуживания:")
print(answer(relative, absolute))

c) Найдите относительную и абсолютную интенсивность обслуживания:
Ответ: 0.9974793376451386, 31.919338804644436



In [17]:
avg_queue_len = average_queue_length(steady_state, m, n)
print("d) Найдите среднюю длину в очереди:")
print(answer(avg_queue_len))

d) Найдите среднюю длину в очереди:
Ответ: 2.016979619710541



In [18]:
avg_queue_time = average_queue_time(steady_state, m, n, mu)
print("e) Найдите среднее время в очереди:")
print(answer(avg_queue_time))

e) Найдите среднее время в очереди:
Ответ: 0.06303061311595441



In [19]:
avg_number_channels = average_number_busy_channels(steady_state, m, n)
print("f) Найдите среднее число занятых каналов:")
print(answer(avg_number_channels))

f) Найдите среднее число занятых каналов:
Ответ: 3.989917350580555



In [20]:
skip_probability = skip_queue_probability(steady_state, m)
print("g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди:")
print(answer(skip_probability))

g) Найдите вероятность того, что поступающая заявка не будет ждать в очереди:
Ответ: 0.4503831726848609



In [21]:
avg_downtime = average_downtime(initial_matrix)
print("h) Найти среднее время простоя системы массового обслуживания:")
print(answer(avg_downtime[0]))

h) Найти среднее время простоя системы массового обслуживания:
Ответ: 0.03125

